In [2]:
%load_ext autoreload
%autoreload 2
# then imported files are u pdated each time a cell is executed (good if functions in py files are added and modified)

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys
import os

from scipy.spatial.transform import Rotation as R
from pyswarms.single.global_best import GlobalBestPSO

# os.add_dll_directory("C:/OpenSim4.4/bin") # otherwise module _sombody not found error!

sys.path.insert(0, '..')
sys.path.insert(0, '...')

from Python.LMC_util import plotLMC, make_LMC, check_LMC_Hand_visibility, make_hand_poses
from Python.plotUtil import plot_hand

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from Python.LMC_util import fingers_idx_LMC

# General Settings
# Data Path
current_dir = os.getcwd()
data_path = 'data/'

# LMC Settings
# Input parameter
finger_radius = 10 # mm

# LMC settings
num_LMC = 1
LMC_H = 400 # mm
LMC_alpha1 = (180-150) / 2 * np.pi / 180
LMC_alpha2 = (180-120) / 2 * np.pi / 180
LMC_position = np.array([0,0,0]) # in mm
LMC_orientation = np.array([0,0,0]) # in deg
LMC = make_LMC(LMC_loc=LMC_position, LMC_orient=LMC_orientation, LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2) 

# PROCES DATA
# load files
for fi, file in enumerate(os.listdir(os.path.join(current_dir, data_path))):
    if fi > 0:
        break
    if 'LMC_' in file and '_Trial' in file:
        print(file)
        # load trial
        filename = os.path.join(current_dir, data_path, file)
        df = pd.read_csv(filename,index_col=False,sep=r',|\t', engine='python')
        df = df.replace('\t','', regex=True)
        #print(df.iloc[[0]].keys())

        # to numpy
        posDatas_ = df.to_numpy()
        posDatas = posDatas_[:, [4, 5, 6] + [13+i for i in range(94-13)]] # cut away unnecessary data
        posDatas = posDatas.reshape(posDatas.shape[0], int(posDatas.shape[1]/3), 3)
        posDatas = posDatas[:, [i for i in range(posDatas.shape[1]) if i not in [0, 2, 3, 8, 13, 18]]] # cut away unnecessary markers

        #print(posDatas.shape)
        # check if labels are correct
        #f = 11
        # frame = (R.from_euler('XYZ', [90, 0, 0], degrees=True).as_matrix().squeeze() @ posDatas[f].T).T
        # print(frame)
        # fig = go.Figure()
        # plot_hand(fig, frame, names=np.arange(0, frame.shape[0]))
        # fig.show()

        occlusions = np.zeros([posDatas.shape[0], 5])
        for f in range(posDatas.shape[0]):  #for each frame
            # rotate data such that z points up
            frame = (R.from_euler('XYZ', [90, 0, 0], degrees=True).as_matrix().squeeze() @ posDatas[f].T).T
            # transform data in global frame not necessary as we only have one LMC
            if np.isnan(frame).any(): # the LMC did not detect the hand:
                occlusions[f, :] = np.array([-1]*5)
                continue
            
            # get data for ray tracing
            handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs = make_hand_poses(np.array([frame]))   
            occlusions_, _ = check_LMC_Hand_visibility(LMC, pos_markers=handMarkers[0], fingers_idx=fingers_idx_LMC, finger_lines=fingers[0], 
                                                        finger_radius=finger_radius, palm_plane_normal=palmPlnNormals[0], 
                                                        palm_centroid=palmCentroids[0], palm_markers=palmMarkers[0], forearm_vec=forearm_vecs[0],
                                                        fig = None, verbose=0)
            occlusions[f, :] = occlusions_[2:]
            #print('    Hand detected: {}'.format(f))
        
        fingers = ['thumb', 'index', 'middle', 'ring', 'pinky']
        for finger in range(5):
            df['visibility_'+fingers[finger]] = occlusions[:, finger]

           

LMC_Naiv_Trial111.txt


d:\Dokumente\Projects\LMC_Optim\Python\LMC_util.py:238: RuntimeWarning:

invalid value encountered in arccos



In [45]:
df.to_csv(os.path.join(current_dir, 'test.txt'), header=df.iloc[[0]].keys(), index=None, sep=' ', mode='a')

In [41]:
print(df.iloca[11])

Device ID            4.000000e+00
Frame ID             4.307000e+03
Frame Rate           1.745920e-02
Time Stamp           1.596278e+10
Palm Position.x               NaN
                         ...     
visibility_thumb    -1.000000e+00
visibility_index    -1.000000e+00
visibility_middle   -1.000000e+00
visibility_ring     -1.000000e+00
visibility_pinky    -1.000000e+00
Name: 11, Length: 99, dtype: float64


# Check finger visibility for a single frame from a given number of LMCs

In [10]:
from Python.LMC_util import fingers_idx_LMC

# Input parameter
finger_radius = 10 # mm

# LMC settings
num_LMC = 1
LMC_H = 600 # mm
LMC_alpha1 = (180-150) / 2 * np.pi / 180
LMC_alpha2 = (180-120) / 2 * np.pi / 180
deviceColor = ['orange', 'blue', 'green', 'black']
deviceLabel = ['Device {}'.format(i) for i in range(1, num_LMC+1)]
LMC_positions = np.array([0,0,0]*num_LMC).reshape(num_LMC, 3) # in mm
LMC_orientations = np.array([0,0,0]*num_LMC).reshape(num_LMC, 3) # in deg
LMCs = [make_LMC(LMC_loc=LMC_positions[i], LMC_orient=LMC_orientations[i], LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2) for i in range(num_LMC)]

# input data
frames = np.array([frame]) # one row for a single frame of each LMC
print(frames.shape)
#transform data in global frame
for lm, LMC in enumerate(LMCs):
    LMC_orient = R.from_euler('xyz', [*LMC[1]], degrees=True).as_matrix().squeeze()
    frames[lm, :] = (LMC_orient @ (frames[lm, :] + LMC[0]).T).T 

# for lm, LMC in enumerate(LMCs):
#     LMC_orient = R.from_euler('xyz', [90,0,-10], degrees=True).as_matrix().squeeze()
#     frames[lm, :] = (LMC_orient @ (frames[lm, :] + LMC[0]).T).T + np.array([0, 200, 300])

# get data for ray tracing
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs = make_hand_poses(frames)   
occlusions = np.zeros([num_LMC, 5])
fig = go.Figure()
for lm, LMC in enumerate(LMCs):
    occlusions_, _ = check_LMC_Hand_visibility(LMC, pos_markers=handMarkers[lm], fingers_idx=fingers_idx_LMC, finger_lines=fingers[lm], 
                                                finger_radius=finger_radius, palm_plane_normal=palmPlnNormals[lm], 
                                                palm_centroid=palmCentroids[lm], palm_markers=palmMarkers[lm], forearm_vec=forearm_vecs[lm],
                                                fig = fig, verbose=1)
    print(occlusions_)
    occlusions[lm] = occlusions_[2:]

print(occlusions) # 0: thumb, 4:pinky


for lm, LMC in enumerate(LMCs):
    fig = plot_hand(fig, frames[lm], color=deviceColor[lm], names=np.arange(0, frames[lm].shape[0]))
    fig = plotLMC(fig, LMC, color=deviceColor[lm], name=deviceLabel[lm], scale=1)
fig.show()

(1, 22, 3)


LinAlgError: SVD did not converge

# Optimize LMC position for Recorded leap data

In [ ]:
from Python.Hand_model import getHandModel, setHandPose, generate_hand_pose, generate_hand_poses, plot_hand
from Python.optimization import PSO_Objective, plotPSOResult
from Python.LMC_util import fingers_idx_LMC

# Input parameter    
finger_radius = 10 # mm

# LMC settings
LMC_H = 600 # mm
num_LMC = 4
LMC_alpha1 = (180-150) / 2 * np.pi / 180
LMC_alpha2 = (180-120) / 2 * np.pi / 180
LMC_positions = np.array([0,0,0]*num_LMC).reshape(num_LMC, 3) # in mm
LMC_orientations = np.array([0,0,0]*num_LMC).reshape(num_LMC, 3) # in deg
LMCs = [make_LMC(LMC_loc=LMC_positions[i], LMC_orient=LMC_orientations[i], LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2) for i in range(num_LMC)]

# input data
'''
  CAREFUL! The optimizer does not work for a single pose! the more the better
          if only a single pose is available then it was to be stacked a few times. e.g 10 times 
          this is caused by the metric that uses an exposnential norm with the number of poses
'''
frames = np.array([frame]*10) # one row for each frame (if the data is from multiple LMC then they are just flattened such that frames.shape = (num_recordedposes*num_LMC, num_markers, 3)
# transform data in global frame
for lm, LMC in enumerate(LMCs):
    LMC_orient = R.from_euler('xyz', [*LMC[1]], degrees=True).as_matrix().squeeze()
    frames[lm, :] = (LMC_orient @ (frames[lm, :] + LMC[0]).T).T 

# get data for ray tracing
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs = make_hand_poses(frames)   
print(handMarkers.shape)
data = [handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs]

# instatiate the optimizer
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
x_max = np.array([1, 1, 1, 1]*num_LMC)
x_min = np.array([-1, -1, -1, -1]*num_LMC)
parameter_mult = np.array([1000, 1000, 90, 180]*num_LMC) 
bounds = (x_min, x_max)

n_particles = 30
num_iterations = 100
args = (data, finger_radius, fingers_idx_LMC, num_LMC, LMC_H, LMC_alpha1, LMC_alpha2, parameter_mult)
optimizer = GlobalBestPSO(n_particles=n_particles, dimensions=4*num_LMC, options=options, bounds=bounds)
# pos is the optimized LMC location and orientation as
'''
  pos = np.array([x, y, xrot, zrot]*num_LMC) / parameter_mult 
  this means pos needs to be multiplied with parameter_mult to get the position in mm and orientation in deg
'''
# a good cost is smaller than 1e-3
cost, pos = optimizer.optimize(PSO_Objective, num_iterations, args=args)
plotPSOResult(pos, args)

# Optimize LMC position for synthetic data

In [ ]:
from Python.Hand_model import getHandModel, setHandPose, generate_hand_pose, generate_hand_poses, plot_hand
from Python.optimization import PSO_Objective, plotPSOResult

# Make a Hand model
HandModel = getHandModel()
q = np.zeros(20)
# set the thumb in a flat position
q[0] = 0.4
q[1] = -0.6
q[2] = 0.1
state = setHandPose(HandModel, q)

# Input parameter 
# these are the finger_idx of the osim model! NOT THE LMC
fingers_idx = np.array([[0,3,8,17], #thumb (root to tip)
                    [4,9,13,18], #index
                    [5,10,14,19], #middle
                    [6,11,15,20], #ring
                    [7,12,16,21]]) #pinky
    
finger_radius = 10 # mm

# LMC settings
LMC_H = 600 # mm
num_LMC = 4
LMC_alpha1 = (180-150) / 2 * np.pi / 180
LMC_alpha2 = (180-120) / 2 * np.pi / 180

# Generate data
'''
  CAREFUL! The optimizer does not work for a single pose! the more the better
          if only a single pose is available then it was to be stacked a few times. e.g 10 times 
          this is caused by the metric that uses an exposnential norm with the number of poses
'''
num_poses = 10
handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs = generate_hand_poses(HandModel, state, num_poses=num_poses, 
                                                                                       x_limits=[0, 0], y_limits=[0, 0], z_limits=[300, 300],
                                                                                         limitsPS=[-70, 70], limis_FE=[-60, 60], limits_WD=[-60, 60])
#handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs = generate_hand_pose(HandModel, state, pos=np.array([0, -100, 300]), PS=90, FE=0, WD=0)
data = [handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs]

# instatiate the optimizer
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
x_max = np.array([1, 1, 1, 1]*num_LMC)
x_min = np.array([-1, -1, -1, -1]*num_LMC)
parameter_mult = np.array([1000, 1000, 90, 180]*num_LMC) 
bounds = (x_min, x_max)

n_particles = 30
num_iterations = 100
args = (data, finger_radius, fingers_idx, num_LMC, LMC_H, LMC_alpha1, LMC_alpha2, parameter_mult)
optimizer = GlobalBestPSO(n_particles=n_particles, dimensions=4*num_LMC, options=options, bounds=bounds)
# pos is the optimized LMC location and orientation as
'''
  pos = np.array([x, y, xrot, zrot]*num_LMC) / parameter_mult 
  this means pos needs to be multiplied with parameter_mult to get the position in mm and orientation in deg
'''
cost, pos = optimizer.optimize(PSO_Objective, num_iterations, args=args)
plotPSOResult(pos, args)